In [1]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/korquad/')
os.getcwd()

'/content/drive/MyDrive/korquad'

In [4]:
!python ./task_1/run_QA.py \
    --model_type distilbert \
    --model_name_or_path distilbert-base-multilingual-cased \
    --output_dir multilingual_distilbert-KMQA \
    --data_dir data \
    --data_name KMQA \
    --train_file KMQA_train_v5.1.json \
    --predict_file KMQA_dev_v5.1.json \
    --max_seq_length 512 \
    --doc_stride 128 \
    --max_query_length 64 \
    --do_train \
    --do_eval \
    --evaluate_during_training \
    --per_gpu_train_batch_size=16 \
    --per_gpu_eval_batch_size=8 \
    --learning_rate 5e-5 \
    --gradient_accumulation_steps 1 \
    --weight_decay 0.0 \
    --adam_epsilon 1e-8 \
    --max_grad_norm 1.0 \
    --num_train_epochs 3 \
    --max_steps -1 \
    --warmup_steps 0 \
    --n_best_size 20 \
    --max_answer_length 30 \
    --verbose_logging \
    --logging_steps 1000 \
    --save_steps 1000 \
    --eval_all_checkpoints \
    --overwrite_output_dir \
    --seed 42 \
    --local_rank -1 \
    --threads 4

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[INFO|squad_metrics.py:305] 2023-06-05 07:16:51,003 >> Unable to find text: '님은 기본적으로 끈기가 있는 분이니 잘 [UNK] 믿습니다' in '님은 기본적으로 끈기가 있는 분이니 잘 견뎌내시리라 믿습니다.조금이라도'
[INFO|squad_metrics.py:305] 2023-06-05 07:16:51,004 >> Unable to find text: '위에서도 언급했듯이 몇 일 가고 말일이 [UNK] . 몇 주에서 수개월 걸릴 수 있습니다' in '위에서도 언급했듯이 몇 일 가고 말일이 아닙니다. 몇 주에서 수개월 걸릴 수 있습니다.5년을'
[INFO|squad_metrics.py:305] 2023-06-05 07:16:51,004 >> Unable to find text: '위에서도 언급했듯이 몇 일 가고 말일이 [UNK]' in '위에서도 언급했듯이 몇 일 가고 말일이 아닙니다.'
[INFO|squad_metrics.py:305] 2023-06-05 07:16:51,005 >> Unable to find text: '비슷한 맥락에서 움직임이 있으면 맥박수가 증가하게 [UNK]' in '비슷한 맥락에서 움직임이 있으면 맥박수가 증가하게 됩니다'
[INFO|squad_metrics.py:305] 2023-06-05 07:16:51,005 >> Unable to find text: '맥박수가 증가하게 [UNK]' in '맥박수가 증가하게 됩니다'
[INFO|squad_metrics.py:305] 2023-06-05 07:16:51,005 >> Unable to find text: '[UNK]' in '됩니다'
[INFO|squad_metrics.py:305] 2023-06-05 07:16:51,006 >> Unable to find text: '##슷한 맥락에서 움직임이 있으면 맥박수가 증가하게 [UNK]' in '비슷한 맥락에서 움직임

# Evaluation

## 경로 변경

In [5]:
import os
os.chdir('/content/drive/MyDrive/korquad/')
os.getcwd()

'/content/drive/MyDrive/korquad'

In [6]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import os

import pandas as pd
from transformers import AutoTokenizer

'''KorQuAD v1.0에 대한 공식 평가 스크립트 '''
'''본 스크립트는 SQuAD v1.1 평가 스크립트 https://rajpurkar.github.io/SQuAD-explorer/ 를 바탕으로 작성됨.'''

def normalize_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
   
    #F1 by character
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)
        
    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)   
        
    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    for article in dataset:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                total += 1
                if qa['id'] not in predictions:
                    message = 'Unanswered question ' + qa['id'] + \
                              ' will receive score 0.'
                    print(message, file=sys.stderr)
                    continue
                ground_truths = list(map(lambda x: x['text'], qa['answers']))
                prediction = predictions[qa['id']]
                exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
                f1 += metric_max_over_ground_truths(
                    f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    return {'exact_match': exact_match, 'f1': f1}


def evaluate_result(dataset_file, prediction_file):
    
    with open(dataset_file, encoding="utf-8") as f:
        dataset_json = json.load(f)
        dataset = dataset_json['data']
    with open(prediction_file, encoding="utf-8") as f:
        predictions = json.load(f)
    return evaluate(dataset, predictions)


## 성능 확인

In [7]:
dataset_file = './data/KMQA_dev_v5.1.json'
prediction_file = './multilingual_distilbert-KMQA/predictions_.json'
evaluate_result(dataset_file, prediction_file)

Unanswered question 413 will receive score 0.
Unanswered question 438 will receive score 0.
Unanswered question 981 will receive score 0.
Unanswered question 1530-0 will receive score 0.
Unanswered question 1530-1 will receive score 0.
Unanswered question 1530-2 will receive score 0.
Unanswered question 1530-3 will receive score 0.
Unanswered question 4311 will receive score 0.
Unanswered question 2339 will receive score 0.
Unanswered question 905 will receive score 0.
Unanswered question 709 will receive score 0.
Unanswered question 2998 will receive score 0.
Unanswered question 2296 will receive score 0.
Unanswered question 1881 will receive score 0.
Unanswered question 86 will receive score 0.
Unanswered question 4132 will receive score 0.
Unanswered question 299 will receive score 0.
Unanswered question 4247 will receive score 0.
Unanswered question 2287 will receive score 0.
Unanswered question 2524 will receive score 0.
Unanswered question 1676 will receive score 0.
Unanswered qu

{'exact_match': 0.9174311926605505, 'f1': 6.673050816554197}